In [1]:
from pprint import *
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB() # making a IB class
# ib.connect('127.0.0.1', 7497, clientId=2)
ib.connect('10.0.0.209', 7497, clientId=1)

<IB connected to 10.0.0.209:7497 clientId=1>

Error 10089, reqId 10: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.NFLX NASDAQ.NMS/TOP/ALL, contract: Stock(conId=15124833, symbol='NFLX', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NFLX', tradingClass='NMS')
Error 10089, reqId 11: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.NFLX NASDAQ.NMS/TOP/ALL, contract: Stock(conId=15124833, symbol='NFLX', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NFLX', tradingClass='NMS')
Error 200, reqId 13: No security definition has been found for the request, contract: Future(symbol='BRR', lastTradeDateOrContractMonth='20240624', exchange='CMECRYPTO')
Error 200, reqId 14: Invalid value in field # 541, contract: Future(symbol='BRR', lastTradeDateOrContractMonth='CMECRYPTO'

In [2]:
nflx_contract = Stock('NFLX', 'SMART', 'USD')
ib.qualifyContracts(nflx_contract)

[Stock(conId=15124833, symbol='NFLX', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NFLX', tradingClass='NMS')]

In [9]:
data = ib.reqMktData(nflx_contract)

In [11]:
data.marketPrice()

nan

In [12]:
eur_usd_contract = Forex('EURUSD', 'IDEALPRO')
ib.qualifyContracts(eur_usd_contract)

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

In [21]:
btc_fut_contract = Future('BRR', '20240426', 'CME')
ib.qualifyContracts(btc_fut_contract)


[Future(conId=662532985, symbol='BRR', lastTradeDateOrContractMonth='20240426', multiplier='5', exchange='CME', currency='USD', localSymbol='BTCJ4', tradingClass='BTC')]

In [ ]:
btc_fut_contract = Future('BRR', 'CME')
ib.qualifyContracts(btc_fut_contract)

In [23]:
gold_cfd_contract = CFD('XAUUSD')
ib.qualifyContracts(gold_cfd_contract)

[CFD(conId=457068913, symbol='XAUUSD', exchange='SMART', currency='USD', localSymbol='XAUUSD', tradingClass='XAUUSD')]

In [24]:
data = ib.reqMktData(gold_cfd_contract)

In [31]:
data.marketPrice()

nan

In [33]:
historical_data_nflx = ib.reqHistoricalData(
    nflx_contract,
    '',
    barSizeSetting='15 mins',
    durationStr='2 D',
    whatToShow='MIDPOINT',
    useRTH=True
)

In [34]:
historical_data_nflx

[BarData(date=datetime.datetime(2024, 4, 1, 9, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=607.49, high=610.67, low=605.53, close=609.39, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 1, 9, 45, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=609.39, high=613.38, low=608.33, close=610.91, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 1, 10, 0, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=610.91, high=611.06, low=608.66, close=610.31, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 1, 10, 15, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=610.31, high=611.88, low=608.88, close=611.66, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 1, 10, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=611.66, high=611.86, low=607.16, close=608.63, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 1, 10, 45, t

In [35]:
type(historical_data_nflx)

ib_insync.objects.BarDataList

In [36]:
historical_data_nflx[-1]

BarData(date=datetime.datetime(2024, 4, 2, 15, 15, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=611.12, high=612.39, low=610.82, close=612.39, volume=-1.0, average=-1.0, barCount=-1)

In [37]:
historical_data_nflx[-1].open

611.12

In [ ]:
util.df(historical_data_nflx)

In [49]:
nflx_order = MarketOrder('BUY',20)

In [50]:
trade = ib.placeOrder(nflx_contract, nflx_order)

In [64]:
trade.log

[TradeLogEntry(time=datetime.datetime(2024, 4, 2, 19, 54, 15, 839422, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 4, 2, 19, 54, 16, 350112, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 4, 2, 20, 1, 25, 423512, tzinfo=datetime.timezone.utc), status='Submitted', message='Fill 20.0@612.68', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 4, 2, 20, 1, 25, 424806, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)]

In [65]:
trade.orderStatus.status

'Filled'

In [66]:
for _ in range(100):
    if not trade.isActive():
        print('Your order status - {0}'.format(trade.orderStatus.status))
        break
    time.sleep(0.5)
else:
    print('Order is still active')

Your order status - Filled


In [67]:
while (trade.isActive()):
    print('Waiting')
    continue
else:
    print('order done')

order done


In [69]:
trade.fills[-1]

Fill(contract=Stock(conId=15124833, symbol='NFLX', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NFLX', tradingClass='NMS'), execution=Execution(execId='00012ec5.660d2dd7.01.01', time=datetime.datetime(2024, 4, 2, 20, 1, 25, tzinfo=datetime.timezone.utc), acctNumber='DU8538876', exchange='NASDAQ', side='BOT', shares=20.0, price=612.68, permId=1535362918, clientId=1, orderId=27, liquidation=0, cumQty=20.0, avgPrice=612.68, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=1), commissionReport=CommissionReport(execId='00012ec5.660d2dd7.01.01', commission=1.0, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 4, 2, 20, 1, 25, 423512, tzinfo=datetime.timezone.utc))

In [80]:
trade.filledEvent

Event<filledEvent, [[None, None, <function order_status at 0x113d705e0>]]>

In [85]:
def order_status(trade):
    if trade.OrderStatus.status == 'Filled':
        fill = trade.fills[-1]
        print('{0} - {1} {2} {3} @ {4}'.format(fill.time, fill.execution.side, fill.contract.symbol, fill.execution.shares, fill.execution.avgPrice))

In [91]:
trade.filledEvent.clear()

In [92]:
trade.filledEvent

Event<filledEvent, []>

In [93]:
trade.filledEvent += order_status

In [94]:
pprint(trade.filledEvent)

Event<filledEvent, [[None, None, <function order_status at 0x113d701f0>]]>


eur_usd_contract = Forex('EURUSD', 'IDEALPRO')
ib.qualifyContracts(eur_usd_contract)

In [95]:
eur_usd_order = ib.bracketOrder(
    'BUY',
    1000,
    limitPrice=1.19,
    takeProfitPrice=1.20,
    stopLossPrice=1.18
)

In [96]:
for ord in eur_usd_order:
    ib.placeOrder(eur_usd_contract, ord)

In [97]:
visa_contract = Stock('V', 'SMART', 'USD')
ib.qualifyContracts(visa_contract)

[Stock(conId=49462172, symbol='V', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='V', tradingClass='V')]

In [98]:
price_condition = PriceCondition(
    price=200,
    conId=visa_contract.conId,
    exch=visa_contract.exchange
)

In [99]:
ma_contract = Stock('MA', 'SMART', 'USD')
ib.qualifyContracts(ma_contract)

[Stock(conId=38685693, symbol='MA', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='MA', tradingClass='MA')]

In [100]:
visa_ma_order = MarketOrder('BUY', 10)
visa_ma_order.conditions.append(price_condition)

In [101]:
visa_ma_trade = ib.placeOrder(ma_contract, visa_ma_order)

In [102]:
visa_ma_trade.isActive()

True